In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from joblib import dump, load
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from IPython.display import display
import io

# 🚀 **1. Tạo widget để tải file CSV**
upload_button = widgets.FileUpload(accept='.csv', multiple=False, description="Tải file CSV")
upload_output = widgets.Output()

categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'thal']
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'ca']
target_column = 'target'

def on_file_upload(change):
    with upload_output:
        upload_output.clear_output()
        if upload_button.value:
            uploaded_file = list(upload_button.value.values())[0]
            content = uploaded_file['content']
            global df, X, y, X_train, X_test, y_train, y_test, preprocessor_loaded, model_loaded
            df = pd.read_csv(io.BytesIO(content))
            print("✅ File CSV đã được tải lên thành công!")
            print("Số hàng trong dữ liệu:", len(df))
            print("Cột trong dữ liệu:", df.columns.tolist())
            print("Dữ liệu mẫu (hàng đầu tiên):")
            print(df.iloc[0])

            X = df[categorical_features + numerical_features]
            y = df[target_column]

            print("5 mẫu có nguy cơ mắc bệnh tim từ dữ liệu gốc:")
            print(df[df[target_column] == 1][categorical_features + numerical_features].head())

            print("Phạm vi giá trị tối đa của các đặc trưng trong các mẫu target=1:")
            for feature in numerical_features:
                print(f"{feature}: max={df[df[target_column] == 1][feature].max()}")

            process_data()

upload_button.observe(on_file_upload, names='value')
display(upload_button, upload_output)

# 🚀 **2. Hàm xử lý dữ liệu và huấn luyện mô hình**
def process_data():
    global X_train, X_test, y_train, y_test, preprocessor_loaded, model_loaded
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    model = RandomForestClassifier(n_estimators=500, max_depth=None, class_weight='balanced', random_state=0)
    model.fit(X_train_transformed, y_train)
    y_pred = model.predict(X_test_transformed)
    print("🎯 Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
    print("📜 Báo cáo phân loại:\n", classification_report(y_test, y_pred))

    print("Dự đoán trên tập kiểm tra (so sánh với thực tế):")
    for i, (pred, actual) in enumerate(zip(y_pred, y_test)):
        print(f"Mẫu {i}: Dự đoán = {pred}, Thực tế = {actual}")

    feature_importance = pd.DataFrame({
        'Feature': preprocessor.get_feature_names_out(),
        'Importance': model.feature_importances_
    })
    print("Độ quan trọng của các đặc trưng:")
    print(feature_importance.sort_values(by='Importance', ascending=False))

    dump(preprocessor, "preprocessor.joblib")
    dump(model, "heart_disease_model.joblib")
    print("✅ Mô hình và bộ tiền xử lý đã được lưu!")
    preprocessor_loaded = load("preprocessor.joblib")
    model_loaded = load("heart_disease_model.joblib")

# 🚀 **3. Tạo giao diện nhập liệu bằng tiếng Việt**
tuoi = widgets.IntText(description="Tuổi:")
gioi_tinh = widgets.Dropdown(options=[("Nữ", 0), ("Nam", 1)], description="Giới tính:")
dau_nguc = widgets.Dropdown(options=[("Đau thắt ngực điển hình", 0), ("Đau thắt ngực không điển hình", 1), ("Đau không phải thắt ngực", 2), ("Không triệu chứng", 3)], description="Loại đau ngực:")
huyet_ap = widgets.FloatText(description="Huyết áp nghỉ (mmHg):")
cholesterol = widgets.FloatText(description="Cholesterol (mg/dl):")
duong_huyet = widgets.Dropdown(options=[("Không", 0), ("Có", 1)], description="Đường huyết cao:")
dien_tam_do = widgets.Dropdown(options=[("Bình thường", 0), ("Bất thường ST-T", 1), ("Phì đại thất trái", 2)], description="Điện tâm đồ nghỉ:")
nhip_tim = widgets.IntText(description="Nhịp tim tối đa:")
dau_nguc_gang_suc = widgets.Dropdown(options=[("Không", 0), ("Có", 1)], description="Đau ngực khi gắng sức:")
so_mach = widgets.IntSlider(min=0, max=4, description="Số mạch bị hẹp:")
thalassemia = widgets.Dropdown(options=[("Không xác định", 1), ("Bình thường", 3), ("Khiếm khuyết cố định", 6), ("Khiếm khuyết đảo ngược", 7)], description="Thalassemia:")

display(tuoi, gioi_tinh, dau_nguc, huyet_ap, cholesterol, duong_huyet, dien_tam_do, nhip_tim, dau_nguc_gang_suc, so_mach, thalassemia)

# 🔥 Nút bấm dự đoán
button = widgets.Button(description="Dự đoán")
output = widgets.Output()

def du_doan_benh_tim(b):
    with output:
        output.clear_output()
        try:
            input_df = pd.DataFrame({
                'age': [tuoi.value],
                'sex': [gioi_tinh.value],
                'cp': [dau_nguc.value],
                'trestbps': [huyet_ap.value],
                'chol': [cholesterol.value],
                'fbs': [duong_huyet.value],
                'restecg': [dien_tam_do.value],
                'thalach': [nhip_tim.value],
                'exang': [dau_nguc_gang_suc.value],
                'ca': [so_mach.value],
                'thal': [thalassemia.value]
            })
            print("Dữ liệu đầu vào để dự đoán:")
            print(input_df.iloc[0])
            input_data = preprocessor_loaded.transform(input_df)
            prediction = model_loaded.predict(input_data)

            # Thêm ngưỡng thủ công cho trường hợp cực đoan



            print("Kết quả dự đoán:", "Có nguy cơ" if prediction[0] == 1 else "Không nguy cơ")
            if prediction[0] == 1:
                print("🔴 Có nguy cơ mắc bệnh tim!")
            else:
                print("🟢 Không có nguy cơ mắc bệnh tim.")
        except NameError:
            print("⚠️ Vui lòng tải file CSV và huấn luyện mô hình trước!")

button.on_click(du_doan_benh_tim)
display(button, output)

FileUpload(value={}, accept='.csv', description='Tải file CSV')

Output()

IntText(value=0, description='Tuổi:')

Dropdown(description='Giới tính:', options=(('Nữ', 0), ('Nam', 1)), value=0)

Dropdown(description='Loại đau ngực:', options=(('Đau thắt ngực điển hình', 0), ('Đau thắt ngực không điển hìn…

...

Dropdown(description='Đau ngực khi gắng sức:', options=(('Không', 0), ('Có', 1)), value=0)

IntSlider(value=0, description='Số mạch bị hẹp:', max=4)

Dropdown(description='Thalassemia:', options=(('Không xác định', 1), ('Bình thường', 3), ('Khiếm khuyết cố địn…

Button(description='Dự đoán', style=ButtonStyle())

Output()